In [0]:
#Data ingestion
%pip install -qU pypdf langchain_community
%pip install --quiet langchain_experimental
%pip install langchain-databricks
%pip install -qU langchain-huggingface
%pip install databricks-vectorsearch
%restart_python

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.9.2 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for langchain-databricks from https://files.pythonhosted.org/packages/a8/db/222105b90d78589cca9deec15e7b13c3f907a7f97c00f53a0da74fa44154/langchain_databricks-0.1.0rc1-py3-none-any.whl.metadata
  Using cached langchain_databricks-0.1.0rc1-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for langchain-core<0.3.0,>=0.2.35 from https://files.pythonhosted.org/packages/e9/20/4d9427db7e2c78b6be6d4300fc22da128c8ca07444d7e23ad

In [0]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
from langchain_databricks import ChatDatabricks
from langchain_databricks import DatabricksEmbeddings

/local_disk0/.ephemeral_nfs/envs/pythonEnv-1bc88354-51a6-421c-96ea-08d2fc2f1ced/lib/python3.11/site-packages/langchain_databricks/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_databricks.chat_models import ChatDatabricks


In [0]:
def process_files(file_path):
    loader=PyPDFLoader(file_path)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    rdocs=[]
    for split in splits:
        rdocs.append(split.page_content)
    return rdocs

In [0]:
file_paths=dbutils.fs.ls("/Volumes/main/default/spectra2/")
rdocs=[]
for file in file_paths:
    docs=process_files(file.path[5:])
    rdocs.append(docs)
rdocs

[['1 1.0 Aims and Objectives of the Scheme  \n \n1.1  To provide holistic and broad based supportive ecosystem to form new 10,000 \nFPOs to facilitate development of vibrant and sustainable income oriented \nfarming and for overall socio -economic development and wellbeing of agrarian \ncommunities.  \n1.2  To enhance productivity through efficient, cost -effective and sustainable resou rce \nuse and realize higher returns through better liquidity and market linkages for \ntheir produce and become sustainable through collective action.  \n1.3  To provide handholding and support to new FPOs up to 5 years from the year of \ncreation in a ll aspects o f management of FPO , inputs, production, processing \nand value addition, market linkages, credit linkages and use of technology etc.  \n1.4 To provide effective capacity building to FPOs to develop agriculture -\nentrepreneurship skills to become economically via ble and self -sustaining \nbeyond the period of support from government.',
  

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("Chunks", StringType(), True) 
])

data = [(i+1,rdocs[i]) for i in range(len(rdocs))]

df = spark.createDataFrame(data, schema)

df.write.format("delta").mode("append").saveAsTable("main.default.spectra3")

display(spark.table("main.default.spectra3"))


id,Chunks
37,"MIS reports/information in the requi red data sheets. s) Assist Implementing Agency and NPMA in rating of FPOs as may be necessary. t) Assist in f ederating FPOs when necessary for business growth and expansion. u) Assist FPO in proper financial management and utilization of fund and accounting and tim ely submission of returns and certificates. v) Any other activity related to implementation, management and monitoring of the project. w) An advisory body consisting of various stakeholders including state government representative, NGOs, RIs, KVKs or any other support institutions will provide active guidance over project implementation. 7.4 Payment to CBBOs : a) Payment will be made to the CBBOs by Implementing Agencies. The Implementing Agencies after applying the due diligence will satisfy themselves with the performance of the CBBOs as per criter ia laid down below at clause (c ) before submitting the proposal to the DAC&FW. The"
38,themselves with the performance of the CBBOs as per criter ia laid down below at clause (c ) before submitting the proposal to the DAC&FW. The Project Management Advisory and Fund Sanctioning Committee (N - PMAFSC) will consider the proposal for release to Implementing Agencies. b) Payment shall be released after receiving the utilization certificate of the previously released amount. c) Following indicative criteria/guidelines are provided for guidance of
39,"13 Implementing Agencies for performance assessmen t of CBBOs - (i) In the first six months of a FPO project, performance of the CBBOs will be assessed based on baseline survey, report submitted and aggregation activities undertaken for FPO formation. (ii) In the period from six months to one year, performance of th e CBBOs will be assessed based on (a) No. of FPOs formed in allocated aspirational districts in different blocks , if any (b) number of farmers mobilized to become the members of FPOs with minimum membership of 300 in case of plain areas and 100 members in case of North -Eastern and Hilly areas ( including such other areas of U Ts) as per prescribed minimum number; (c) Registration of FPOs ; and (d) formulation of Business Plan for FPOs. (iii) In the second year, performance will be based on (a) first equity grant availed by the FPOs;(b) statutory clearances obtained to carry out business activities ; (c) minimum 50% of business activities executed"
40,"availed by the FPOs;(b) statutory clearances obtained to carry out business activities ; (c) minimum 50% of business activities executed as per business plan ; (d) number of preliminary awareness programmes for member/BoDs and exposure visits of FPOs are undertaken ; (e) first tranche of Equity Gra nt availed by FPOs, if any ; (f) first tranche of Credit Guarantee Facility availed by FPOs, if any ; (g) Institutional Training provided to CEOs/BoDs ; and (h) registration of FPOs in e -NAM or other electronic platform undertaken and trading activity thereon taken place . (iv) In third and fourth year, performance of the CBBOs wi ll be assessed based on – (a) issuing Share Certificates to each member in third year, if any ; (b) audited Financial Statements for FPOs for second year and third year in due time and filing as required ; (c) MoU and vendor registration as per Business Plan with Marketing Agencies/Institutional"
41,"and third year in due time and filing as required ; (c) MoU and vendor registration as per Business Plan with Marketing Agencies/Institutional Buyers ; (d) trading/uploading of produce in e -NAM/other sources, if any; (e) second tranche equ ity grant to FPOs, if any ; and (f) second tranche of credit guarantee facility, if any . (v) In the fi fth year, performance of the CBBOs will be assessed based on (a) audited Statements of accounts of FPO a nd filing it; (b) 100% of agri-business plan executed and v alue chain developed; (c) revenue model showing financial growth in last 3 consecutive years ; (d) detailed project completion Report ; and (e) t

In [0]:
%sql
ALTER TABLE `main`.`default`.`spectra3` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
from databricks.vector_search.client import VectorSearchClient
client = VectorSearchClient()
client.create_endpoint(
    name="spectra3",
    endpoint_type="STANDARD"
)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


{'name': 'spectra2',
 'creator': 'hackathon_ai2@centific.com',
 'creation_timestamp': 1726488886711,
 'last_updated_timestamp': 1726488886711,
 'endpoint_type': 'STANDARD',
 'last_updated_user': 'hackathon_ai2@centific.com',
 'id': 'ffae8ccb-0aaf-4bf4-9b9f-409dd96463ef',
 'endpoint_status': {'state': 'PROVISIONING'},
 'num_indexes': 0}

In [0]:
%sql
ALTER TABLE `main`.`default`.`spectra3` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
client = VectorSearchClient(disable_notice=True)
index = client.create_delta_sync_index(
  endpoint_name="spectra2",
  source_table_name="main.default.Spectra3",
  index_name="main.default.Spectra3_index",
  pipeline_type="CONTINUOUS",
  primary_key="id",
  embedding_source_column="Chunks",
  embedding_model_endpoint_name="databricks-gte-large-en"
)


In [0]:
import os
from databricks.vector_search.client import VectorSearchClient
workspace_url = os.environ.get("WORKSPACE_URL")
sp_client_id = os.environ.get("SP_CLIENT_ID")
sp_client_secret = os.environ.get("SP_CLIENT_SECRET")
vsc = VectorSearchClient(
    workspace_url=workspace_url,
    service_principal_client_id=sp_client_id,
    service_principal_client_secret=sp_client_secret
)
question = "what are the services of FPO"
results = vsc.get_index("spectra3","main.default.spectra3_index").similarity_search(
  query_text=question,
  columns=["Chunks"],
  num_results=3)
rest = results.get('result', {}).get('data_array', [])
rest

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


[['process and day today management through suitable provisioning in their \nMemorandum of Association and Bye -laws with a view to encourage healthy \ngrowth and development of FPO.  \n3.0  Broad Services and A ctivities to be undertaken by FPOs  \n \nThe FPOs may provide and undertake following relevant major services and \nactivities for their development as may be necessary: -  \n(i) Supply quality production inputs like seed, fertilizer, pesticides and such other \ninputs at rea sonably lower wholesale rates.  \n(ii) Make available need based production and post -production machinery and \nequipment like cultivator, tiller, sprinkler set, combine harvester and such other \nmachinery and equipment on custom hiring basis for members to reduce th e per',
  0.0033626081]]

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": "databricks-dbrx-instruct",
    "vector_search_endpoint_name": "spectra3",  # the endoint we want to use for vector search
    "vector_search_index":"main.default.spectra3_index",
    "llm_prompt_template": """You are an assistant that answers questions. use the following pieces of retrieved context to answer the question. Some pieces of context may be irrelevant, in which case you should not use them to form the answer.Look between the sentences for answers because answers might not be explicitly mentioned.\n\nContext: {context}""",
}

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks
from operator import itemgetter

prompt = ChatPromptTemplate.from_messages(
    [  
        ("system", chain_config.get("llm_prompt_template")), 
        ("user", "{question}") 
    ]
)

model = ChatDatabricks(
    endpoint=chain_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)
answer = (prompt | model).invoke({'question': question, 'context': rest})
answer.content

'Based on the provided context, FPOs (Farmers Producer Organizations) can provide and undertake the following major services and activities for their development:\n\n1. Supply quality production inputs: FPOs can supply quality inputs like seeds, fertilizers, pesticides, and other necessary inputs at reasonably lower wholesale rates.\n2. Make available need-based production and post-production machinery and equipment: FPOs can provide machinery and equipment like cultivators, tillers, sprinkler sets, combine harvesters, and other machinery and equipment on a custom hiring basis for members to reduce the per-unit cost of production.\n\nThese services are aimed at encouraging healthy growth and development of FPOs and their members.'